# SORA SCRAPER

In [42]:
import requests
from bs4 import BeautifulSoup
from pathlib import Path

# — Configuration & setup (as before) —
url     = "https://eservices.mas.gov.sg/statistics/dir/DomesticInterestRates.aspx"
session = requests.Session()
hdrs    = {"User-Agent":"Mozilla/5.0","Referer":url}

# 1) Grab the form + hidden fields
r1   = session.get(url, headers=hdrs, timeout=10)
soup = BeautifulSoup(r1.text, "html.parser")
def hidden(n): return soup.find("input",{"name":n})["value"]

payload = {
  "__EVENTTARGET":   "",
  "__EVENTARGUMENT": "",
  "__VIEWSTATE":          hidden("__VIEWSTATE"),
  "__VIEWSTATEGENERATOR": hidden("__VIEWSTATEGENERATOR"),
  "__EVENTVALIDATION":    hidden("__EVENTVALIDATION"),
  "ctl00$ContentPlaceHolder1$StartYearDropDownList":  "2019",
  "ctl00$ContentPlaceHolder1$StartMonthDropDownList": "1",
  "ctl00$ContentPlaceHolder1$EndYearDropDownList":    "2025",
  "ctl00$ContentPlaceHolder1$EndMonthDropDownList":   "7",
}

# 2) Build a map of label→(input_name, value)
checkboxes = soup.select("input[type=checkbox]")
series_map = {}
for cb in checkboxes:
    cb_id    = cb["id"]
    label    = soup.find("label", {"for": cb_id}).text.strip()
    name     = cb["name"]
    # ASP.NET default for unchecked without a value is “on”
    value    = cb.get("value", "on")
    series_map[label] = (name, value)

# 3) Tell the script which series you want
to_select = [
    "SORA",
    "SORA Index",
    "1-month Compounded SORA"
]

# 4) Add those checkboxes to your payload
for label in to_select:
    if label not in series_map:
        raise KeyError(f"Series not found on page: {label}")
    name, val = series_map[label]
    payload[name] = val

# 5) Indicate you clicked “Download”
payload["ctl00$ContentPlaceHolder1$Button2"] = "Download"

# 6) POST & save CSV (overwrites old file)
r2 = session.post(url, headers=hdrs, data=payload, timeout=30)
r2.raise_for_status()

csv_path = Path.cwd() / "DomesticInterestRates.csv"
with open(csv_path, "wb") as f:
    f.write(r2.content)

print(f"✅ Downloaded & overwrote: {csv_path}")


✅ Downloaded & overwrote: /Users/isaac/Downloads/DomesticInterestRates.csv


# SGS Scraper


In [46]:
import requests
from bs4 import BeautifulSoup
from pathlib import Path

# 1. Configuration
url     = "https://eservices.mas.gov.sg/statistics/fdanet/SgsBenchmarkIssuePrices.aspx"
session = requests.Session()
headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": url
}

# 2. GET the page and parse hidden ASP.NET fields
r1 = session.get(url, headers=headers, timeout=10)
r1.raise_for_status()
soup = BeautifulSoup(r1.text, "html.parser")

def hidden(name):
    el = soup.find("input", {"name": name})
    return el["value"] if el else ""

payload = {
    "__EVENTTARGET":        "",
    "__EVENTARGUMENT":      "",
    "__VIEWSTATE":          hidden("__VIEWSTATE"),
    "__VIEWSTATEGENERATOR": hidden("__VIEWSTATEGENERATOR"),
    "__EVENTVALIDATION":    hidden("__EVENTVALIDATION"),
}

# 3. Find the Download button and add its name/value to the payload
#    (the button is <button name="ctl00$ContentPlaceHolder1$btnDownload">Download</button>)
download_btn = soup.find(
    lambda tag: tag.name in ("button","input")
                and ((tag.name=="button" and tag.text.strip().lower()=="download")
                     or (tag.name=="input" and tag.get("value","").lower()=="download"))
)
btn_name  = download_btn["name"]
btn_value = download_btn.get("value", download_btn.text.strip())
payload[btn_name] = btn_value

# 4. POST back to get the CSV bytes
r2 = session.post(url, headers=headers, data=payload, timeout=30)
r2.raise_for_status()

# 5. Verify CSV & write to disk
content_type = r2.headers.get("Content-Type","")
if "text/csv" not in content_type:
    raise RuntimeError(f"Expected CSV but got {content_type}")

out_file = Path.cwd() / "SgsBenchmarkIssuePrices.csv"
out_file.write_bytes(r2.content)
print(f"✅ Downloaded and saved to {out_file}")


✅ Downloaded and saved to /Users/isaac/Downloads/SgsBenchmarkIssuePrices.csv
